In [76]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

names = pd.read_csv("merged_city_data_normalized.csv", encoding = 'ISO-8859-1')
names = names[['GeoID', 'Place Name']]
names = names.drop_duplicates()

combo = pd.read_csv("merged_city_data_normalized_with_employment.csv", encoding = 'ISO-8859-1')
combo_original = pd.read_csv("merged_city_data_normalized.csv", encoding = 'ISO-8859-1')
combo_original = combo_original.drop_duplicates(subset = ['GeoID'])
print(combo_original)

aff = pd.read_csv("Affiliate-City-to-Id2.csv", encoding = 'ISO-8859-1')
aff = aff[['Geoid']]
aff.columns = ['GeoID']
aff['Affiliate'] = pd.Series(['Yes']*160)


      Unnamed: 0    GeoID                        Place Name  Indeed job count  \
0              5   100820           Alabaster city, Alabama             622.0   
2              7   100988         Albertville city, Alabama             143.0   
3              8   101132      Alexander City city, Alabama              67.0   
5             14   101708           Andalusia city, Alabama              16.0   
6             16   101852            Anniston city, Alabama             114.0   
7             17   102116                Arab city, Alabama             180.0   
8             25   102956              Athens city, Alabama             452.0   
9             26   103004              Atmore city, Alabama              23.0   
10            28   103076              Auburn city, Alabama             164.0   
11            37   104660         Bay Minette city, Alabama             215.0   
12            48   105980            Bessemer city, Alabama             729.0   
14            51   107000   

In [44]:
data = combo.drop(['GeoID'], axis = 1)
#data = data.drop(['Farmers Markets per 1000 people (2016) min_max_normalized'], axis = 1)
#data = data.drop(['Farmer Markets with SNAP per 1000 people (2016) min_max_normalized'], axis = 1)
#data = data.drop(['Farmer Markets with SFMNP per 1000 people (2016) min_max_normalized'], axis = 1)
#data = data.drop(['Farmer Markets with WIC Cash per 1000 people (2016) min_max_normalized'], axis = 1)

#data['Unemployment rate min_max_normalized'] = 1-data['Unemployment rate min_max_normalized']
#data['Number of people below the poverty level normalized min_max_normalized'] = 1-data['Number of people below the poverty level normalized min_max_normalized']
#data = data.drop(['Unemployment rate min_max_normalized'], axis = 1)
#data = data.drop(['Number of people below the poverty level normalized min_max_normalized'], axis = 1)
arr = data.values
print(data.columns.values)

['Population min_max_normalized' 'Unemployment rate min_max_normalized'
 'Public transportation proportion min_max_normalized'
 'Simpsons diversity score min_max_normalized'
 'Affordable housing in market min_max_normalized'
 'Proportion of public transit to work under 30 minutes min_max_normalized'
 'SNAP-authorized Stores per 1000 people (2016) min_max_normalized'
 'Farmers Markets per 1000 people (2016) min_max_normalized'
 'Farmer Markets with SNAP per 1000 people (2016) min_max_normalized'
 'Farmer Markets with WIC per 1000 people (2016) min_max_normalized'
 'Farmer Markets with WIC Cash per 1000 people (2016) min_max_normalized'
 'Farmer Markets with SFMNP per 1000 people (2016) min_max_normalized'
 'Employment to population ratio percentage min_max_normalized'
 'Indeed job count normalized min_max_normalized'
 'Craigslist affordable house count normalized min_max_normalized'
 'Number of people below the poverty level normalized min_max_normalized'
 'Number of people with a bache

In [0]:
#bandwidth = estimate_bandwidth(arr, quantile=0.2, n_samples=500)
bandwidth = 0.35

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(arr)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique, counts = np.unique(labels, return_counts=True)
countClusters = dict(zip(labels_unique, counts))
n_clusters_ = len(labels_unique)

In [105]:
res = pd.DataFrame(combo['GeoID'])
res['Cluster'] = pd.Series(labels)

res = res.merge(names, on = 'GeoID')
res = res.merge(aff, how = 'left', on = 'GeoID')

print(countClusters)
print(labels)
print(labels_unique)
print(res)
#print(cluster_centers)

df2 = pd.DataFrame(cluster_centers)
print(df2)

df2.to_csv("cluster_center.csv")

{0: 1811, 1: 1664, 2: 1262, 3: 37, 4: 3, 5: 7, 6: 14, 7: 118, 8: 6, 9: 25, 10: 1, 11: 37, 12: 2, 13: 1, 14: 27, 15: 1, 16: 11, 17: 17, 18: 76, 19: 10, 20: 11, 21: 1, 22: 29, 23: 1, 24: 17, 25: 6, 26: 2, 27: 70, 28: 13, 29: 1, 30: 95, 31: 1, 32: 44, 33: 7, 34: 113, 35: 1, 36: 3, 37: 135, 38: 1, 39: 3, 40: 16, 41: 39, 42: 33, 43: 11}
[ 0  2 34 ...  1  2  2]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
        GeoID  Cluster                        Place Name Affiliate
0      100820        0           Alabaster city, Alabama       NaN
1      100988        2         Albertville city, Alabama       NaN
2      101132       34      Alexander City city, Alabama       NaN
3      101708        0           Andalusia city, Alabama       NaN
4      101852       41            Anniston city, Alabama       NaN
5      102116        0                Arab city, Alabama       NaN
6      102956        0              At

In [103]:
cluster1 = pd.DataFrame((res.loc[res['Cluster'] == 0])['GeoID'])

cluster2 = pd.DataFrame((res.loc[res['Cluster'] == 1])['GeoID'])

cluster3 = pd.DataFrame((res.loc[res['Cluster'] == 2])['GeoID'])

cluster1 = cluster1.merge(combo_original, on = 'GeoID', how = 'left')
cluster2 = cluster2.merge(combo_original, on = 'GeoID', how = 'left')
cluster3 = cluster3.merge(combo_original, on = 'GeoID', how = 'left')

df3 = pd.DataFrame(cluster1.mean())
df3["Cluster 1 Standard Dev"] = pd.Series(cluster1.std())
print(cluster1.mean())
print(cluster1.std())
df3["Cluster 2 Mean"] = pd.Series(cluster2.mean())
df3["Cluster 2 Standard Deviation"] = pd.Series(cluster2.std())
df3["Cluster 3 Mean"] = pd.Series(cluster3.mean())
df3["Cluster 3 Standard Deviation"] = pd.Series(cluster3.std())

print(df3)

df3.to_csv('res2.csv')


GeoID                                                                               3.067936e+06
Unnamed: 0                                                                          1.570122e+04
Indeed job count                                                                    2.091956e+02
Craigslist affordable house count                                                   6.738824e+01
Population                                                                          1.503280e+04
Unemployment rate                                                                   5.470391e+00
Employment to population ratio                                                      6.022943e+01
Number of people below the poverty level                                            8.078955e+02
Number of people with a bachelors degree or higher                                  2.625056e+03
Public transportation proportion                                                    1.475577e-02
Simpsons diversity score      

In [48]:
res1 = res.loc[res['Cluster'] == 3]
print(res1)

res2 = res.loc[res['Cluster'] == 6]
print(res2)

res4 = res.loc[res['Cluster'] == 7]
print(res4)

res5 = res.loc[res['Cluster'] == 14]
print(res5)

res6 = res.loc[res['Cluster'] == 15]
print(res6)

res7 = res.loc[res['Cluster'] == 16]
print(res7)

res8 = res.loc[res['Cluster'] == 18]
print(res8)

res9 = res.loc[res['Cluster'] == 21]
print(res9)

        GeoID  Cluster                          Place Name Affiliate
306    606000        3           Berkeley city, California       NaN
628    653000        3            Oakland city, California       Yes
2453  2416875        3               Chillum CDP, Maryland       NaN
2495  2433250        3              Glenmont CDP, Maryland       NaN
2503  2441250        3          Hyattsville city, Maryland       NaN
2532  2453325        3    Montgomery Village CDP, Maryland       NaN
2534  2454275        3        Mount Rainier city, Maryland       NaN
2535  2455400        3       New Carrollton city, Maryland       NaN
2590  2483775        3               Wheaton CDP, Maryland       NaN
2606  2507000        3          Boston city, Massachusetts       NaN
2642  2537875        3          Malden city, Massachusetts       NaN
3274  3403580        3            Bayonne city, New Jersey       NaN
3300  3413570        3  Cliffside Park borough, New Jersey       NaN
3316  3420020        3       Edgew

In [49]:
affCluster = res.loc[res['Affiliate'] == 'Yes']
print(affCluster)

counts = affCluster['Cluster'].value_counts()
print(counts)

        GeoID  Cluster                             Place Name Affiliate
55     150000        1                   Mobile city, Alabama       Yes
100    203000        1         Anchorage municipality, Alaska       Yes
168    455000        8                  Phoenix city, Arizona       Yes
194    477000        1                   Tucson city, Arizona       Yes
253    566080        2              Springdale city, Arkansas       Yes
274    602000        1               Anaheim city, California       Yes
458    629000        1          Garden Grove city, California       Yes
460    630000        1              Glendale city, California       Yes
563    644000        8           Los Angeles city, California       Yes
565    644112        1             Los Gatos town, California       Yes
596    648354        1               Modesto city, California       Yes
628    653000        3               Oakland city, California       Yes
716    664000        1            Sacramento city, California   

Counts for Clusters:{0: 1889, 1: 1699, 2: 1300, 3: 37, 4: 3, 5: 7, 6: 14, 7: 120, 8: 3, 9: 1, 10: 3, 11: 41, 12: 2, 13: 1, 14: 23, 15: 1, 16: 2, 17: 17, 18: 78, 19: 31, 20: 14, 21: 11, 22: 1, 23: 24, 24: 1, 25: 21, 26: 6, 27: 2, 28: 17, 29: 1, 30: 1, 31: 31, 32: 8, 33: 99, 34: 1, 35: 3, 36: 165, 37: 1, 38: 3, 39: 18, 40: 39, 41: 33, 42: 11}

Counts  for Affiliate Clusters: 

1     97
2     29
18     9
7      7
14     4
10     3
8      3
3      2
40     1
28     1
19     1
11     1
9      1
0      1